In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/apple/Documents/Personal/Computer Vision and Deep Learning/pc-parts'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
  root_dir: Path
  untrained_base_model: Path
  trained_base_model: Path
  classes: int

In [5]:
from pcpartsclassifier.utils.common import read_yaml, create_directories
from pcpartsclassifier.constants import *

class ConfigurationManager:
  def __init__(self,
               config = CONFIG_FILE_PATH,
               params = PARAMS_FILE_PATH) -> None:
    self.config = read_yaml(config)
    self.params = read_yaml(params)
    create_directories([self.config.artifacts_root])
    
  def get_base_model_config(self) -> BaseModelConfig:
    config = self.config.base_model
    base_model_config = BaseModelConfig(
      root_dir=config.root_dir,
      untrained_base_model=config.untrained_base_model,
      trained_base_model=config.trained_base_model,
      classes = self.params.CLASSES
    )
    return base_model_config


In [9]:
from torchvision import models
import torch.nn as nn
import torch


class BaseModel(nn.Module):
  def __init__(self,  config: BaseModelConfig):
    super(BaseModel, self).__init__()
    self.config = config
    self.weights = models.EfficientNet_B0_Weights.DEFAULT
    self.transform = self.weights.transforms()
    self.model = models.efficientnet_b0(weights=self.weights)

    for params in self.model.features.parameters():
      params.requires_grad = False

    self.model.classifier = nn.Sequential(
      nn.Dropout(),
      nn.Linear(1280, self.config.classes)
    )

  def forward(self, x):
    return self.model(x)
  
  def save_untrained_model(self) -> None:
    create_directories([self.config.root_dir])
    torch.save(self.model.state_dict(), self.config.untrained_base_model)

  @staticmethod
  def save_model(model:nn.Module, path: Path) -> None:
    torch.save(model.state_dict(), path)


In [10]:
# from torchinfo import summary
# config = ConfigurationManager()
# base_model_config = config.get_base_model_config()
# model = BaseModel(base_model_config)
# model

# summary(model, input_size=[1, 3, 224, 224], col_names=["input_size",  
#                                                         "output_size",
#                                                         "num_params",
#                                                         "kernel_size",  
#                                                         "trainable"])



In [11]:
# pipeline
try:
  config = ConfigurationManager()
  base_model_config = config.get_base_model_config()
  base_model = BaseModel(config=base_model_config)
  base_model.save_untrained_model()
except Exception as e:
  raise e

[2024-03-30 12:47:45,006 : INFO : common : yaml file: config/config.yaml loaded successfull]
[2024-03-30 12:47:45,038 : INFO : common : yaml file: params.yaml loaded successfull]
[2024-03-30 12:47:45,080 : INFO : common : created directory at artifacts]
[2024-03-30 12:47:45,368 : INFO : common : created directory at artifacts/base_model]
